In [ ]:
import boto3
import botocore
import pyarrow.parquet as pq
import pandas as pd

## Download a Parquet file from S3
See https://kafka-connect-manager.lsst.io/ for more information on the S3 Sink connector and references on it. For this example the S3 Sink connector is configure to partition data by time on an hourly basis.

In [ ]:
BUCKET_NAME = "efd-sandbox.data"
KEY = "topics/example-002-aggregated/year=2020/month=08/day=06/hour=22/example-002-aggregated+0+0000022314.snappy.parquet"
#KEY = "topics/example-000-aggregated/year=2020/month=08/day=06/hour=16/example-000-aggregated+0+0000000177.snappy.parquet"

S3 credentials are added to `~/.aws/credentials`file and the S3 region to the`~/.aws/config` file as explained here https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html

In [ ]:
s3 = boto3.resource('s3')
s3.Bucket(BUCKET_NAME).download_file(KEY, 'example-002-aggregated+0+0000022314.snappy.parquet')

## Use Pyarrow read the Parquet file 

In [ ]:
example_002_aggregated = pq.read_table('example-002-aggregated+0+0000022314.snappy.parquet')

## Convert from Parquet to Pandas Dataframe

In [ ]:
df = example_002_aggregated.to_pandas()
df.head()

## Plotting the aggregated stream

In [ ]:
p = df.plot(x='time', y='mean_value1', c='white', figsize=(15,5))
p.fill_between(x='time', y1='min_value1', y2='max_value1', data=df)